In [1]:
import os
import requests
from bs4 import BeautifulSoup
import json 
from dotenv import load_dotenv
from typing import List
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key wrking fine")
else:
    print("Check api key again")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key wrking fine


In [3]:
header={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
}

class Website:
    def __init__(self,url):
        self.url=url
        response=requests.get(url, headers=header)
        self.body=response.content
        soup=BeautifulSoup(self.body, 'html.parser')
        self.title=soup.title.string if soup.title else "No title found"
        if soup.body:
            for faaltu in soup.body(['script', 'style','img','input']):
                faaltu.decompose()
            self.text=soup.body.get_text(separator='\n', strip=True)
        else:
            self.text=""
        links=[link.get('href') for link in soup.find_all('a')]
        self.links=[link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
system_prompt="You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"

system_prompt+='You should respond in JSON as in this example:'
system_prompt+="""
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_user_prompt(website):
    user_prompt=f'Here is the list of links on the website of {website.url} - '
    user_prompt+="please decide which of these are relevant web links  for a brochure about the company, respond with the full https URL in JSON format. \
        do not include Terms of Service, Privacy, email links.\n"
    user_prompt+="Links (some might be relative links):\nGive the output strictly in python list form"
    user_prompt+="\n".join(website.links)

    return user_prompt

In [6]:
def get_links(url):
    website=Website(url)
    response=openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content":system_prompt},
            {"role":"user", "content": get_user_prompt(website)}
        ],

            response_format={"type":"json_object"}
    )
    result=response.choices[0].message.content
    return json.loads(result)
    

In [13]:
import time

def retry(url, retries=5, delay=2):
    for attempt in range(retries):
        try:
            return Website(url).get_contents()  
        except (ConnectTimeout, TimeoutError, Exception) as e:
            print(f"Error occurred for {url}: {e}. Attempt {attempt + 1} of {retries}.")
            time.sleep(delay)  
    print(f"Skipping {url} after {retries} failed attempts.")
    return f"Failed to retrieve contents for {url}\n"

def get_details(url):
    result = "Landing page:\n"
    result += retry(url)  
    links = get_links(url)
    print("Found links:", links)
    
    key_to_check = next(iter(links), None) 
    
    if key_to_check == 'links':
        for link in links["links"]:
            result += f"\n\n{link}\n"
            result += retry(link)  
    elif key_to_check == 'relevant_links':
        for link in links["relevant_links"]:
            result += f"\n\n{link}\n"
            result += retry(link)  
    else:
        result += "\nNo relevant links found.\n"
    
    return result


In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a website ( it can be company website, college club website, someone's portfolio) so be prepared \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [15]:
def brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a professional brief brochure of the website in markdown.At footer always give the contacts link of their social media handles (except phone number) if available\n"
    user_prompt += get_details(url)
    user_prompt = user_prompt[:5_000] 
    return user_prompt

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("Riviera", "https://www.anthropic.com/")

Found links: {'relevant_links': ['https://www.anthropic.com/claude', 'https://www.anthropic.com/team', 'https://www.anthropic.com/enterprise', 'https://www.anthropic.com/api', 'https://www.anthropic.com/pricing', 'https://www.anthropic.com/research', 'https://www.anthropic.com/company', 'https://www.anthropic.com/careers', 'https://www.anthropic.com/news', 'https://www.anthropic.com/jobs', 'https://support.anthropic.com/', 'https://status.anthropic.com/', 'https://twitter.com/AnthropicAI', 'https://www.linkedin.com/company/anthropicresearch', 'https://www.youtube.com/@anthropic-ai', 'https://trust.anthropic.com/']}


# Welcome to Anthropic: A Front-Row Seat to a Safer AI Future! 🎉

### Meet Claude: Your New AI BFF 🤖✨
Say hello to **Claude**—not just any AI, but *the* AI that moonlights as your trusty co-pilot on the journey to creativity and productivity! Whether you're a lone wolf or part of a bustling team, Claude's here to help you turn ideas into reality. Why? Because we believe that every bright mind deserves a little quirky virtual assistant in their corner!

### What’s in Claude’s Toolbox? 🛠️
- **Advanced Reasoning:** Forget about simple patty-flipping; this AI can hold its own in complex cognitive tasks. 
- **Vision Analysis:** Claude can even make sense of your what-on-earth-is-this scribbles and *that* blurry selfie you took at brunch!
- **Multilingual Processing:** Say “hola” to your multilingual skills! If you’ve ever wondered how to say "delicious" in ten languages, Claude’s got you covered!
- **Code Generation:** It’s like having a mini-coder in your pocket—build websites, debug like a pro, and transform images into structured data faster than you can say “syntax error!”

### Our Values: Safety is the Name of the Game ⚖️
At Anthropic, our primary focus is on **AI safety**. We believe that technology should protect and serve humanity—like a superhero wearing a tie! Expect top-notch data security, user-friendly integrations, and policies that ensure Claude is not just smart, but *trustworthy*.

### Calling All Visionaries! 🌟 Join Our Team!
Want to become part of our quirky family? At Anthropic, our culture celebrates creativity, diversity, and the *occasional* AI-related dad joke. Check out our **open roles** because there’s no ‘I’ in AI, but there’s definitely a ‘we’!

### Who's Building with Claude? 📈
Our **A-listers** include *leading companies* across various sectors relying on our AI to revolutionize their workflows. Think of us as the “cutting-edge” wingman for your business!

### Ready to Try Claude? 🚀
Jump in and experience the future! From brainstorming sessions to heavy-duty tasks, Claude is all set to maximize your efficiency and fun factor.

### Let’s Connect! 🌐
Got questions? Want to know how we can light up your business life? Hit us up on our social media:
- [Twitter](https://twitter.com/Anthropic)
- [LinkedIn](https://www.linkedin.com/company/anthropic)
- [YouTube](https://www.youtube.com/@Anthropic)

**Dive into the thrilling world of Anthropic—where AI safety meets creative genius!**